In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:

import os
for dirname, _, filenames in os.walk('../data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../data\anime.csv
../data\rating.csv


In [4]:
import pandas_summary as ps
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate, KFold, GridSearchCV
from surprise import SVD, NMF, SlopeOne, KNNBasic, KNNWithMeans, KNNBaseline, CoClustering, BaselineOnly, NormalPredictor
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
import datetime
from operator import itemgetter
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.prediction_algorithms.matrix_factorization import SVDpp, SVD


In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [7]:
folder = '../data/'
anime_df = pd.read_csv(folder+'anime.csv')
rate_df = pd.read_csv(folder+'rating.csv')

In [6]:
print('anime: ', anime_df.shape)
print('rating: ', rate_df.shape)

anime:  (12294, 7)
rating:  (7813737, 3)


In [8]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [9]:
full_df = rate_df.merge(anime_df, how='left', left_on=['anime_id'], right_on=['anime_id'])
df = full_df[full_df['rating_x'] != -1]
df.head()

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
47,1,8074,10,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892.0
81,1,11617,10,High School DxD,"Comedy, Demons, Ecchi, Harem, Romance, School",TV,12,7.70,398660.0
83,1,11757,10,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100.0
101,1,15451,10,High School DxD New,"Action, Comedy, Demons, Ecchi, Harem, Romance,...",TV,12,7.87,266657.0
153,2,11771,10,Kuroko no Basket,"Comedy, School, Shounen, Sports",TV,25,8.46,338315.0


In [10]:
dfs = ps.DataFrameSummary(full_df)
print('categoricals: ', dfs.categoricals.tolist())
print('numerics: ', dfs.numerics.tolist())
dfs.summary()

categoricals:  ['name', 'genre', 'type', 'episodes']
numerics:  ['user_id', 'anime_id', 'rating_x', 'rating_y', 'members']


,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
count,7813737.0,7813737.0,7813737.0,NaN,NaN,NaN,NaN,7813721.0,7813727.0
mean,36727.956745,8909.072104,6.14403,NaN,NaN,NaN,NaN,7.653127,178620.804981
std,20997.946119,8883.949636,3.7278,NaN,NaN,NaN,NaN,0.673029,188176.417151
min,1.0,1.0,-1.0,NaN,NaN,NaN,NaN,1.67,29.0
25%,18974.0,1240.0,6.0,NaN,NaN,NaN,NaN,7.27,44030.0
50%,36791.0,6213.0,7.0,NaN,NaN,NaN,NaN,7.68,110470.0
75%,54757.0,14093.0,9.0,NaN,NaN,NaN,NaN,8.13,244268.0
max,73516.0,34519.0,10.0,NaN,NaN,NaN,NaN,9.5,1013917.0
counts,7813737,7813737,7813737,7813727,7813617,7813723,7813727,7813721,7813727
uniques,73515,11200,11,11196,3154,6,184,585,6487


In [11]:
min_anime_ratings = 250
filter_anime = df['anime_id'].value_counts() > min_anime_ratings
filter_anime = filter_anime[filter_anime].index.tolist()

min_user_ratings = 250
filter_users = df['user_id'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['anime_id'].isin(filter_anime)) & (df['user_id'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))
df_new

The original data frame shape:	(6337241, 9)
The new data frame shape:	(2368822, 9)


,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
302,5,6,8,Trigun,"Action, Comedy, Sci-Fi",TV,26,8.32,283069.0
303,5,15,6,Eyeshield 21,"Action, Comedy, Shounen, Sports",TV,145,8.08,83648.0
304,5,17,6,Hungry Heart: Wild Striker,"Comedy, Shounen, Slice of Life, Sports",TV,52,7.74,13469.0
305,5,18,6,Initial D Fourth Stage,"Action, Cars, Drama, Seinen, Sports",TV,24,8.24,41584.0
306,5,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297.0
...,...,...,...,...,...,...,...,...,...
7813329,73507,8211,5,Dance in the Vampire Bund Recap,"Action, Supernatural, Vampire",Special,1,6.61,9758.0
7813331,73507,8348,5,Tengen Toppa Gurren Lagann: Parallel Works 2,"Mecha, Music",Music,7,7.09,13361.0
7813332,73507,8440,7,Black Lagoon Omake,Comedy,Special,7,7.02,33923.0
7813333,73507,8769,8,Ore no Imouto ga Konnani Kawaii Wake ga Nai,"Comedy, Seinen, Slice of Life",TV,12,7.49,321477.0


In [12]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df_new[['user_id', 'anime_id', 'rating_x']], reader)
data

In [13]:
algo = SVD(biased=False)
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)



{'test_rmse': array([1.08680414, 1.08698493, 1.08565772, 1.0859979 , 1.08790055]),
 'fit_time': (42.9102246761322,
  54.62861180305481,
  44.71566462516785,
  61.5068244934082,
  58.54916048049927),
 'test_time': (12.38604211807251,
  9.806036472320557,
  12.418980360031128,
  15.543078422546387,
  13.923075914382935)}

In [14]:
def hyperTune(data, modelTarget, leaveOneOut=False):
    if modelTarget == "als":
        param_grid = {"bsl_options": {"n_epochs": epochs, "reg_u": regUs, "reg_i": regIs, "method": ["als"]}}
    else:
        param_grid = {"bsl_options": {"n_epochs": epochs, "learning_rate": lRates, "method": ["sgd"]}}

    if leaveOneOut:
        gs = GridSearchCV(BaselineOnly, param_grid, measures=["rmse", "mae"], cv=LeaveOneOut())

    else:
        gs = GridSearchCV(BaselineOnly, param_grid, measures=["rmse", "mae"], cv=5)

    gs.fit(data)

    print("RMSE score", gs.best_score['rmse'], " with ", gs.best_params)

    return (gs.best_score['mae'], gs.best_params['rmse'])

In [14]:
# trainset, testset = train_test_split(data, test_size=0.25)
# randAlgorith = random_pred.NormalPredictor()
# randAlgorith.fit(trainset)


In [15]:
method = ["als", "sgd"]
epochs = [5, 10]
regUs = [5, 10, 15, 20]
regIs = [5, 10, 15, 20]
lRates = [0.05, 0.005, 0.0005]

setOfValue = [hyperTune(data, "als"),  hyperTune(data, "sgd")]
print("best model : "+str(min(setOfValue,key=itemgetter(0))[1]))


Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimati

In [16]:
cv = KFold(n_splits=5, random_state=777)
cvSlope = KFold(n_splits=2, random_state=777)
targets = [
           {"name": "Co-clustering", "param_grid": {"n_epochs": [20, 30], "n_cltr_u": [3, 6], "n_cltr_i": [3, 6]},"verbose": [False], "model": CoClustering, "CV": cv},
           {"name":"Slope one", "param_grid": {}, "verbose": [False], "model": SlopeOne, "CV":cvSlope},
           {"name":"knn basic","param_grid":{"sim_options": {"name": ["msd"], "min_support":[2,5,10]}, "k": [30,40], "min_k": [1,10], "verbose": [False], "user_based": [True, False]},"model":KNNBasic, "CV":cv},
           {"name":"Random Base", "param_grid":{}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Baseline ALS", "param_grid":{"bsl_options": {"n_epochs": [5, 10], "reg_u": [5, 10, 20], "reg_i": [5, 10, 20], "method": ["als"]}}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Baseline SGD", "param_grid":{"bsl_options": {"n_epochs": [5, 10],  "learning_rate": [0.005, 0.0005], "reg": [0.02, 0.01], "method": ["sgd"]}}, "verbose": [False], "model":BaselineOnly, "CV":cv},
           {"name":"Matrix factorization based", "param_grid":{"lr_all":[0.007, 0.01, 0.15]}, "verbose": [False], "model":SVD, "CV":cv},
           {"name":"Matrix factorization based + implicit rating", "param_grid":{"lr_all":[0.007, 0.01, 0.15]}, "verbose": [False], "model":SVDpp, "CV":cv}
]

In [ ]:
targetedMetric = "mae"
bestScoringModel = [np.inf, None, None]


LOOCV = LeaveOneOut(n_splits=1, random_state=777)
train_loocv, test_loocv  = LOOCV.split(data).__next__()
for target in targets:
    print("****** doing ", target)
    np.random.seed(40)
    scoringModel, modelParam, model = hyperTune( target["param_grid"], data, target["model"], targetedMetric, target["CV"])
    print("best mae for ", target["name"], ":", scoringModel, " with parameters :", target["param_grid"])
    if scoringModel < bestScoringModel[0]:
        print("found new best tuned model !")
        bestScoringModel[0] = scoringModel
        bestScoringModel[1] = modelParam
        bestScoringModel[2] = model

    get_hitrate_results(bestScoringModel[2](), train_loocv, test_loocv, 10, 2)
    print("************************")


get_hitrate_results(bestScoringModel[2](), train_loocv, test_loocv, 10, 2)